In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from google.colab import files
import io


In [2]:
def trainDataSet():
    uploaded = files.upload()
    df = pd.read_excel(io.BytesIO(uploaded['train.xls']))
    try:
        df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
        df.drop(["a"], axis=1, inplace=True)
        df = df.drop([0], axis=0, inplace=False)
        df.duplicated().sum()
        df = df.drop_duplicates()
    except:
        df = df
    
    arr = df.to_numpy()
    X = arr[:, 0:23]
    Y = arr[:,23]
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=42)
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train=scaler.transform(X_train)
    X_test=scaler.transform(X_test)
    
    return (X_train, X_test, Y_train, Y_test, scaler)


In [3]:
def currentDataSet(data):
    X_train, X_test, Y_train, Y_test, scaler = trainDataSet()
    X_input = data.to_numpy()
    X_input = scaler.transform(X_input)
    return X_input


In [4]:
(X_train, X_test, Y_train, Y_test, scaler) = trainDataSet()

Saving train.xls to train (4).xls


In [5]:
p=np.sum(Y_train)
total = np.shape(Y_train)[0]
weight0 = (1/(total-p)*total/2.0)
weight1 = ((1/p)*total/2.0)
class_weights = { 0: weight0, 1: weight1}

In [6]:
X_train = tf.convert_to_tensor(X_train, dtype=tf.float64)
Y_train = tf.convert_to_tensor(Y_train, dtype=tf.float64)
X_test = tf.convert_to_tensor(X_test, dtype=tf.float64)
Y_test = tf.convert_to_tensor(Y_test, dtype=tf.float64)

In [7]:
tf.shape(X_train)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([22473,    23], dtype=int32)>

In [28]:
from tensorflow import keras
from tensorflow.keras import layers
model = keras.Sequential(
    [ keras.Input(shape=(23,)),
      layers.Dense(15, activation = "sigmoid"),
      layers.Dense(8, activation = "sigmoid"),
      layers.Dense(6, activation = "sigmoid"),
      layers.Dense(3, activation = "sigmoid"),
      layers.Dense(1, activation = "sigmoid", name = "output")])


In [30]:
model.compile(
     optimizer=keras.optimizers.Adam(),
     loss=keras.losses.BinaryCrossentropy(),
     metrics=[keras.metrics.TruePositives(), keras.metrics.FalseNegatives(), keras.metrics.FalsePositives(), keras.metrics.TrueNegatives(), keras.metrics.Recall(), keras.metrics.BinaryAccuracy(), keras.metrics.AUC()])

In [31]:
history = model.fit(X_train, Y_train, epochs = 50  , class_weight = class_weights)

Epoch 1/50
703/703 [==============================] - 4s 3ms/step - loss: 0.6792 - true_positives_6: 2003.0000 - false_negatives_6: 2940.0000 - false_positives_6: 2916.0000 - true_negatives_6: 14614.0000 - recall_1: 0.4052 - binary_accuracy: 0.7394 - auc_6: 0.6297
Epoch 2/50
703/703 [==============================] - 2s 3ms/step - loss: 0.6246 - true_positives_6: 2720.0000 - false_negatives_6: 2223.0000 - false_positives_6: 3251.0000 - true_negatives_6: 14279.0000 - recall_1: 0.5503 - binary_accuracy: 0.7564 - auc_6: 0.7117
Epoch 3/50
703/703 [==============================] - 3s 4ms/step - loss: 0.6009 - true_positives_6: 2577.0000 - false_negatives_6: 2366.0000 - false_positives_6: 2528.0000 - true_negatives_6: 15002.0000 - recall_1: 0.5213 - binary_accuracy: 0.7822 - auc_6: 0.7210
Epoch 4/50
703/703 [==============================] - 3s 4ms/step - loss: 0.5950 - true_positives_6: 2561.0000 - false_negatives_6: 2382.0000 - false_positives_6: 2456.0000 - true_negatives_6: 15074.0000 -

In [32]:
results = model.evaluate(X_test, Y_test)

235/235 [==============================] - 2s 3ms/step - loss: 0.5480 - true_positives_6: 972.0000 - false_negatives_6: 715.0000 - false_positives_6: 993.0000 - true_negatives_6: 4812.0000 - recall_1: 0.5762 - binary_accuracy: 0.7720 - auc_6: 0.7679
